In [6]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
%matplotlib inline

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images = x_train[0:1000] / 255
labels = y_train[0:1000]
one_hot_labels = np.zeros((len(labels), 10))
for i, j in enumerate(labels):
    one_hot_labels[i][j] = 1
labels = one_hot_labels

test_images = x_test / 255
test_labels = np.zeros((len(y_test), 10))
for i, j in enumerate(y_test):
    test_labels[i][j] = 1

np.random.seed()
alpha = 0.005
max_iteration = 30
rank_kernel_conv = 3
layer_3_size = 26 * 26
layer_4_size = 100
layer_5_size = 10

relu = lambda x: (x > 0) * x
relu_deriv = lambda x: x >= 0

weights_kernel_conv_1_2 = 0.2 * np.random.random((rank_kernel_conv, rank_kernel_conv)) - 0.1
weights_layer_3_4 = 0.2 * np.random.random((layer_3_size, layer_4_size)) - 0.1
weights_layer_4_5 = 0.2 * np.random.random((layer_4_size, layer_5_size)) - 0.1

for iteration in range(max_iteration):
    error_prediction = 0
    correct_cnt = 0
    
    for image_number in range(len(images)):
        layer_0 = images[image_number]
        
        rank_layer_conv_1 = layer_0.shape[1] - rank_kernel_conv + 1
        layer_conv_1 = np.zeros((rank_layer_conv_1, rank_layer_conv_1))
        
        for y in range(rank_layer_conv_1):
            for x in range(rank_layer_conv_1):
                cell = np.zeros((rank_kernel_conv, rank_kernel_conv))
                for j in range(rank_kernel_conv):
                    for i in range(rank_kernel_conv):
                        cell[i][j] = layer_0[x + i][y + j]
                layer_conv_1[x][y] = relu(cell.dot(weights_kernel_conv_1_2).sum())

        layer_3 = layer_conv_1.reshape(1, len(layer_conv_1) ** 2)
        layer_4 = relu(np.dot(layer_3, weights_layer_3_4))
        dropout_mask = np.random.randint(2, size = layer_4.shape)
        layer_4 *= dropout_mask * 2
        layer_5 = np.dot(layer_4, weights_layer_4_5)        
        
        error_prediction += np.sum((labels[image_number:image_number+1] - layer_5) ** 2)
        correct_cnt += int(np.argmax(layer_5)) == np.argmax(labels[image_number:image_number+1])
        
        layer_5_delta = (labels[image_number:image_number+1] - layer_5)
        layer_4_delta = layer_5_delta.dot(weights_layer_4_5.T) * relu_deriv(layer_4)
        layer_4_delta *= dropout_mask
        layer_3_delta = layer_4_delta * relu_deriv(layer_3)
        for y in range(layer_3_delta.shape[1]):
            for x in range(rank_layer_conv_1):
                cell = np.zeros((rank_kernel_conv, rank_kernel_conv))
                for j in range(rank_kernel_conv):
                    for i in range(rank_kernel_conv):
                        cell[i][j] = layer_0[x + i][y + j]
                layer_conv_1[x][y] = relu(cell.dot(weights_kernel_conv_1_2).sum())
        
        #weights_layer_4_5 += alpha * layer_4.T.dot(layer_5_delta)
        #weights_layer_3_4 += alpha * layer_3.T.dot(layer_4_delta)
       

ValueError: operands could not be broadcast together with shapes (1,100) (1,676) 